## Install pyodbc
Did that using conda


## Import libraries

Database details

In [1]:

from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from tornado.options import options

# set spark configs details
conf = (SparkConf()
        .setAppName("ETLPipeline")
        .setMaster("local")
        .set("spark.driver.extraClassPath", "G:/pyspark/*"))

# Initiate Spark Session
sc = SparkContext.getOrCreate(conf=conf)
etl = SparkSession(sc)

# Set Database details


24/10/16 02:24:01 WARN Utils: Your hostname, haiduri-Legion-5-Pro-16ACH6H resolves to a loopback address: 127.0.1.1; using 192.168.50.104 instead (on interface wlp4s0)
24/10/16 02:24:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/16 02:24:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import pyodbc
import pandas as pd
from IPython.display import display
from sqlalchemy import create_engine, text
import os

# Connection parameters
server_sqlserver = 'localhost'
port_sqlserver = '1433'
database_sqlserver = 'AdventureWorksDW2019'
username_sqlserver = 'sa'
password_sqlserver = 'YourStrong!Passw0rd'



server_ps = 'localhost'
port_ps = '5433'
database_ps = 'AdventureWorks'
username_ps = 'root'
password_ps = 'root'


# connection_string_sqlserver = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server_sqlserver},{port_sqlserver};DATABASE={database_sqlserver};UID={username_sqlserver};PWD={password_sqlserver}' # for pyodbc

connection_string_sqlserver = f'mssql+pyodbc://{username_sqlserver}:{password_sqlserver}@{server_sqlserver}:{port_sqlserver}/{database_sqlserver}?driver=ODBC+Driver+17+for+SQL+Server'  # for sqlalchemy




# Connection string for PostgreSQL
connection_string_ps = (
    f'postgresql+psycopg2://{username_ps}:{password_ps}'
    f'@{server_ps}:{port_ps}/{database_ps}'
)

server = "localhost"
src_driver="com.microsoft.sqlserver.jdbc.SQLServerDriver"
src_db = "AdventureWorksDW2019"
src_url=f"jdbc:sqlserver://{server}:1433;database={src_db};user={username_sqlserver};password={password_sqlserver};encrypt=true;trustServerCertificate=true"

target_driver= "org.postgresql.Driver"
target_db = "AdventureWorks"
target_url = f"jdbc:postgresql://{server}:5433/{target_db}?user={username_ps}&password={password_ps}"


In [3]:
try:
    engine_sqlserver = create_engine(connection_string_sqlserver)  # for sqlalchemy
    # src_conn = pyodbc.connect(connection_string_sqlserver)
    print("Connected to SqlServer successfully!\n")    
    
    engine_ps = create_engine(connection_string_ps)
    print("Connected to PostgreSQL successfully!\n")
except Exception as e:
    print("Error: ", e)

Connected to SqlServer successfully!

Connected to PostgreSQL successfully!



In [4]:
sql = text("""
            SELECT t.name AS table_name
            FROM sys.tables t 
            WHERE t.name IN ('DimProduct', 'DimProductSubCategory', 'DimProductCategory', 'DimSalesTerritory', 'FactInternetSales')
        """)


In [5]:
dfs=etl.read.format("jdbc").options(driver=src_driver,user=username_sqlserver,password=password_sqlserver,url=src_url,query=sql).load()
dfs.show()

+--------------------+
|          table_name|
+--------------------+
|          DimProduct|
|  DimProductCategory|
|DimProductSubcate...|
|   DimSalesTerritory|
|   FactInternetSales|
+--------------------+



In [6]:
data_collect = dfs.collect()
for row in data_collect:
    print(row["table_name"])

DimProduct
DimProductCategory
DimProductSubcategory
DimSalesTerritory
FactInternetSales


# ETL Process
- extract: get data from a source
- transform: structure, format, or clean the data
- load: write the data to an external destination

In [7]:
def extract():
    try:
        dfs=etl.read.format("jdbc").options(driver=src_driver,user=username_sqlserver,password=password_sqlserver,url=src_url,query=sql).load()
        
        data_collect=dfs.collect()
        
        for row in data_collect:
            print(row["table_name"])
            tbl_name= row["table_name"]
            df=(etl.read.format("jdbc")
                .option("driver", src_driver)
                .option("user", username_sqlserver)
                .option("password", password_sqlserver)
                .option("url", src_url)
                .option("dbtable",f"dbo.{tbl_name}")
                .load())
            print(df.show(10))
            load(df,tbl_name)
            print("Data loaded successfully")
    except Exception as e:
        print("Error: ", str(e))

In [8]:
def load(df, tbl):
    try:
        rows_imported = 0

        print(f'importing rows {rows_imported} to {rows_imported + df.count()}... for table {tbl}')

        df.write.mode("overwrite") \
         .format("jdbc") \
            .option("url", target_url) \
            .option("user", username_ps) \
            .option("password", password_ps) \
            .option("driver", target_driver) \
            .option("dbtable", "src_" + tbl) \
            .save()

        print("data imported successfully")

        rows_imported += df.count()

    except Exception as e:
        print("Error:", str(e))

In [9]:
try:
    extract()
except Exception as e:
    print("Error:", str(e))

DimProduct


24/10/16 02:24:06 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+-------------------+---------------------+---------------------+-------------------+--------------------+------------------+-----------------+------------+-----------------+------+----------------+------------+---------+----+---------+------+-----------------+-----------+-----------+-----+-----+---------+--------------------+------------------+-----------------+------------------+-----------------+-----------------+---------------+-----------------+-------------------+------------------+-------------------+-------+-------+
|ProductKey|ProductAlternateKey|ProductSubcategoryKey|WeightUnitMeasureCode|SizeUnitMeasureCode|  EnglishProductName|SpanishProductName|FrenchProductName|StandardCost|FinishedGoodsFlag| Color|SafetyStockLevel|ReorderPoint|ListPrice|Size|SizeRange|Weight|DaysToManufacture|ProductLine|DealerPrice|Class|Style|ModelName|          LargePhoto|EnglishDescription|FrenchDescription|ChineseDescription|ArabicDescription|HebrewDescription|ThaiDescription|GermanDescri

data imported successfully
Data loaded successfully
